In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import re

In [2]:
data = pd.read_json('salary_pred.json', encoding="UTF8")

In [3]:
data.head()

,_id,job_title,company,job_location,salary,summary,post_date
0,3d5487bc57a59ba4,nouveauData Scientist Big Data (F/H),AXA,Nanterre (92),,Gestion de projet data et data science. Animat...,Postedil y a 1 jour
1,de372ba90e161d50,nouveauData Analyst H/F,ACII,37000 Tours,35 000 € - 45 000 € par an,Une société participative et collaborative ave...,Postedil y a 6 jours
2,42b16956665ad440,Data Analyst,Daher,Tours (37),,Vous avez une appétence pour la data analyse e...,Postedil y a 30+ jours
3,f4faf36bbce2cda4,Data analyst campagne publicitaire Pinterest à...,Teleperformance Portugal,France,,"En tant qu’analyste publicitaire, votre rôle s...",Postedil y a 30+ jours
4,a17e6a7d7f9adb2e,Data Analyst F/H,AUDENSIEL ASSURANCES,Hybrid remote in Tours (37),,33 - 42 k€ brut annuel. Nous sommes à la reche...,Postedil y a 26 jours


In [4]:
#data description
data.describe()

,_id,job_title,company,job_location,salary,summary,post_date
count,5179,5179,5179,5179,5179,5179,5179
unique,5179,4367,2367,923,326,4747,60
top,3d5487bc57a59ba4,Business Developer H/F,Data Recrutement,Paris (75),,"Cette Talent Company rassemble Designers, Stra...",Postedil y a 30+ jours
freq,1,47,70,1391,4444,13,2600


In [5]:
#Data Informations
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5179 entries, 0 to 5178
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           5179 non-null   object
 1   job_title     5179 non-null   object
 2   company       5179 non-null   object
 3   job_location  5179 non-null   object
 4   salary        5179 non-null   object
 5   summary       5179 non-null   object
 6   post_date     5179 non-null   object
dtypes: object(7)
memory usage: 283.4+ KB


In [6]:
data.nunique()

_id             5179
job_title       4367
company         2367
job_location     923
salary           326
summary         4747
post_date         60
dtype: int64

In [7]:
#Checking missing values

data.isnull().sum()

_id             0
job_title       0
company         0
job_location    0
salary          0
summary         0
post_date       0
dtype: int64

In [8]:
#Dropping the NULL Values from Column, 'Salaire'

data = data.dropna()
data.shape

(5179, 7)

### Texte processing

### Lowerization

In [9]:
data = data.applymap(lambda s: s.lower() if type(s) == str else s)

### Salary processing: Minimum, maximum et mean

salary pay type: par heure, par jour, par semaine, par mois, par an

In [10]:
#Creating columns for each salay pay type
data['per_hour'] = data['salary'].apply(lambda x: 1 if 'par heure' in x.lower() else 0 )

In [11]:
data['per_day'] = data['salary'].apply(lambda x: 1 if 'par jour' in x.lower() else 0 )

In [12]:
data['per_week'] = data['salary'].apply(lambda x: 1 if 'par semaine' in x.lower() else 0 )

In [13]:
data['per_month'] = data['salary'].apply(lambda x: 1 if 'par mois' in x.lower() else 0 )

In [14]:
data['per_year'] = data['salary'].apply(lambda x: 1 if 'par an' in x.lower() else 0 )

In [64]:
salary_range = data['salary'].apply(lambda x:
                                   x.replace('€', '')
                                   .replace('par heure', '')
                                   .replace('par an', '')
                                   .replace('par mois', '')
                                   .replace('par semaine', '')
                                   .replace('par jour', '')
                                   .replace('18,14', '')
                                   .replace(' ','')).astype(str)
salary_range.describe()

count     5179
unique     321
top           
freq      4445
Name: salary, dtype: object

In [35]:
data['min_sal'] = salary_range.apply(lambda x: x.split('-')[0])

In [36]:
#bar graph

data.sort_values('salary', inplace=True)

fig = px.histogram(data, x='min_sal')
#fig.show()

In [41]:
data['max_sal'] = salary_range.apply(lambda x: x.split('-')[1] if len(x.split('-'))>1 else x)

In [42]:
#bar graph

data.sort_values('salary', inplace=True)

fig = px.histogram(data, x='Max_sal')
#fig.show()

In [44]:
#df[df['min_sal'] + df['max_sal']]
data['mean_sal'] = (pd.to_numeric(data['min_sal']) + pd.to_numeric(data['Max_sal']))/2

In [45]:
#Monthly salary
m1 = data['salary'].str.contains('par heure')
m2 = data['salary'].str.contains('par jour')
m3 = data['salary'].str.contains('par semaine') #par mois is a default value
m5 = data['salary'].str.contains('par an')

data['monthly_salary'] = np.select([m1, m2, m3, m5],
                                  [data['mean_sal']*152, #converting hourly to monthly
                                   data['mean_sal']*22,   #converting daily to monthly                               
                                   data['mean_sal']*4,    #weeklyly to monthly
                                   data['mean_sal']/12],   #monthly to monthly
                                   default=data['mean_sal']) #default value
data['monthly_salary'].tail().round()

4751     8750.0
4749    10000.0
1396      950.0
1687     1050.0
4227      900.0
Name: monthly_salary, dtype: float64

In [48]:
import numpy as np

#Annual salary

m1 = data['salary'].str.contains('par heure')
m2 = data['salary'].str.contains('par jour')
m3 = data['salary'].str.contains('par semaine')
m4 = data['salary'].str.contains('par mois')

data['Annual_salary'] = np.select([m1, m2, m3, m4],
                                  [data['mean_sal']*1825, #converting hourly to annual
                                   data['mean_sal']*260,   #converting daily to annual                               
                                   data['mean_sal']*52,    #weeklyly to annual
                                   data['mean_sal']*12],   #monthly to annual
                                   default=data['mean_sal']) #default value
data['Annual_salary'].describe()

count       734.000000
mean      49604.126022
std       27284.574683
min        1560.000000
25%       36500.000000
50%       45000.000000
75%       57500.000000
max      264000.000000
Name: Annual_salary, dtype: float64

In [49]:
#bar graph

data.sort_values('salary', inplace=True)

fig = px.histogram(data, x='Annual_salary')
#fig.show()

In [50]:
import regex

In [51]:
data['job_location'] = data['job_location'].apply(lambda x: x.split('.')[0])

In [52]:
data['job_location'] = data['job_location'].apply(lambda x: x
                                    .replace('télétravail', '')
                                    .replace('temporaire', '')
                                    .replace('hybrid remote', '')
                                    .replace('in', '')
                                    .replace('+ lieu', '')
                                    .replace('+lieu', '')
                                    .split('.')[0])
data['job_location'].head(5)

0       nanterre (92)
1098                 
1103       91000 évry
1172        nice (06)
1171      angers (49)
Name: job_location, dtype: object

In [55]:
location_data = data['job_location'].replace(to_replace=r'.\d+.|[+]', value='', regex=True)

In [60]:
data['Location'] = location_data.apply(lambda x: x.split('-')[0]).str.capitalize()
data['Location'].describe()

count       5179
unique       610
top       Paris 
freq        1538
Name: Location, dtype: object

Reference: 

1. <a href="https://www.geeksforgeeks.org/get-all-rows-in-a-pandas-dataframe-containing-given-substring/">Get all rows in a Pandas DataFrame containing</a>

2. <a href="https://nycdatascience.com/blog/student-works/job-salary-prediction-with-python/">Job Salary Prediction with Python</a>

3. <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html">pandas.Series.replace</a>

In [61]:
data.head()

,_id,job_title,company,job_location,salary,summary,post_date,per_hour,per_day,per_week,per_month,per_year,min_sal,max_sal,Max_sal,mean_sal,monthly_salary,Annual_salary,Location
0,3d5487bc57a59ba4,nouveaudata scientist big data (f/h),axa,nanterre (92),,gestion de projet data et data science. animat...,postedil y a 1 jour,0,0,0,0,0,,,NaN,NaN,NaN,NaN,Nanterre
1098,8fc0751b0de97506,solutions engineering manager,cloudera,,,"present roadmap, vision, proposed architecture...",postedil y a 10 jours,0,0,0,0,0,,,NaN,NaN,NaN,NaN,
1103,e594c23cf8a37774,business developer h/f,hierø,91000 évry,,contribution à la recherche de nouveaux parten...,postedil y a 12 jours,0,0,0,0,0,,,NaN,NaN,NaN,NaN,Évry
1172,f437f4daa465984d,systems engineer,oticon,nice (06),,if you thrive with analyzing and structuring s...,postedil y a 30+ jours,0,0,0,0,0,,,NaN,NaN,NaN,NaN,Nice
1171,006b0f53d6963313,administrateur des ventes h/f,eolane,angers (49),,"les 177 collaborateurs (opérateurs, technicien...",postedil y a 19 jours,0,0,0,0,0,,,NaN,NaN,NaN,NaN,Angers
